In [1]:
# %pip install xhtml2pdf
# %pip install plotly==5.10.0

In [2]:
# %pip install -U kaleido

In [3]:
from IPython.display import display, HTML
from xhtml2pdf import pisa 
from dotenv import load_dotenv
from sqlalchemy import create_engine
from datetime import datetime, timedelta
from os import getenv
import pandas as pd
import plotly.express as px
import io
from base64 import b64encode
import os

In [4]:
load_dotenv()

True

In [5]:
db_host = getenv('DB_HOST')
db_port = getenv('DB_PORT')
db_user = getenv('DB_USER')
db_password = getenv('DB_PASSWORD')
db_name = getenv('DB_NAME')
group_user = getenv('GROUP_USER')
group_user_pass = getenv('GROUP_USER_PASS')

In [6]:
engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')

In [7]:
con = engine.connect()

In [8]:
def get_current_date() -> str:
   """
   Returns the current date (note :currently set as yesterdays date)
   """
   current_date = str(datetime.now().date() - timedelta(days = 1))
   return current_date
current_date = get_current_date()
current_date

'2022-10-08'

In [9]:
query = """
SELECT * FROM yusra_stories_production.rides
"""
pd.read_sql_query(query, con)

,ride_id,user_id,start_time,end_time,total_duration,max_heart_rate_bpm,min_heart_rate_bpm,avg_heart_rate_bpm,avg_resistance,avg_rpm,total_power_kilojoules
0,1,4688,2022-10-08 17:43:13,2022-10-08 17:51:54,0:08:40,150,78,109,38,50,10.15
1,2,4688,2022-10-08 17:51:56,2022-10-08 18:00:35,0:08:38,155,70,113,38,50,9.49
2,3,4688,2022-10-08 18:00:36,2022-10-08 18:09:17,0:08:39,140,78,103,38,54,12.12
3,4,4688,2022-10-08 18:09:18,2022-10-08 18:17:59,0:08:40,180,83,132,38,45,14.86
4,5,4688,2022-10-08 18:18:01,2022-10-08 18:26:42,0:08:40,140,82,112,38,46,9.32
5,6,4688,2022-10-08 18:26:44,2022-10-08 18:35:25,0:08:40,146,79,109,38,46,10.06
6,7,4688,2022-10-08 18:35:26,2022-10-08 18:44:07,0:08:40,140,80,93,38,49,11.43


### Quering database

#### Number of rides completed in the past day

In [10]:
query = f"""
WITH rides AS (  
    SELECT *, CAST(start_time AS DATE) AS start_date
    FROM yusra_stories_production.rides
    )
SELECT COUNT(*) AS number_of_rides 
FROM rides 
WHERE start_date = '{current_date}';
"""
number_of_rides = pd.read_sql_query(query, con)
number_of_rides

,number_of_rides
0,7


#### Gender split of riders of the past day

In [11]:
query = f"""
WITH rides AS (  
    SELECT *, CAST(start_time AS DATE) AS start_date
    FROM yusra_stories_production.rides
    ),
riders AS (
SELECT DISTINCT (user_id), name, gender, age
FROM yusra_stories_production.users
JOIN rides
USING (user_id)
WHERE start_date = '{current_date}'
)
SELECT gender, COUNT(*) AS number_of_riders
FROM riders
GROUP BY gender;
"""
riders_gender_split = pd.read_sql_query(query, con)
riders_gender_split_fig = px.pie(riders_gender_split, values='number_of_riders', names='gender', title=f'Gender split of riders of the past day', color_discrete_sequence=px.colors.sequential.Greens_r)
riders_gender_split_fig

#### Ages of the riders of the past day

In [27]:
query = f"""
WITH rides AS (  
    SELECT *, CAST(start_time AS DATE) AS start_date
    FROM yusra_stories_production.rides
    )
SELECT DISTINCT (user_id), name, gender, age
FROM yusra_stories_production.users
JOIN rides
USING (user_id)
WHERE start_date = '{current_date}'
ORDER BY age ASC
"""
ages_of_riders = pd.read_sql_query(query, con)
px.bar()
ages_of_riders

OperationalError: (psycopg2.OperationalError) server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.
server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.
server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.

[SQL: 
WITH rides AS (  
    SELECT *, CAST(start_time AS DATE) AS start_date
    FROM yusra_stories_production.rides
    )
SELECT DISTINCT (user_id), name, gender, age
FROM yusra_stories_production.users
JOIN rides
USING (user_id)
WHERE start_date = '2022-10-08'
ORDER BY age ASC
]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

#### Average power and heart rate of riders of past day

In [13]:
query = f"""
WITH rides AS (  
    SELECT *, CAST(start_time AS DATE) AS start_date
    FROM yusra_stories_production.rides
    )
SELECT user_id, AVG(avg_heart_rate_bpm) AS average_heart_rate, AVG(total_power_kilojoules) AS average_power
FROM yusra_stories_production.users
JOIN rides
USING (user_id)
WHERE start_date = '{current_date}'
GROUP BY user_id
"""
riders_average_power_and_heart_rate = pd.read_sql_query(query, con)
riders_average_power_and_heart_rate

,user_id,average_heart_rate,average_power
0,4688,110.142857,11.061429


### Saving fig as image

#### Option 2: saving to directory

In [17]:
# def create_directory_for_images():
#     if not os.path.exists("images"):
#         os.mkdir("images")
# create_directory_for_images()

In [18]:
# def save_image_as_png(fig, fig_name):
#     fig.write_image(f"images/{fig_name}.png")
# save_image_as_png(riders_gender_split_fig, 'riders_gender_split_fig')

### Create the HTML Template

In [25]:
def report_block_template(fig_name, caption=''):

    graph_block =  (''
            
                '<img style="height: 400px;" src="images/{fig_name}.png">'
           )

    report_block = ('' +
        graph_block +
        '{caption}' + 
        '<br>'      + 
        '</a>' +
        '<br>' +
        '<hr>')                       
    report_layout = (
       '<h2>Deloton Exercise Bikes Daily Report</h2>'
       + '<hr>'
       + report_block
    )
    return report_layout.format(fig_name=fig_name, caption=caption)


report = report_block_template('riders_gender_split_fig', caption='Gender split')


In [26]:
display(HTML(report))

### Convert the HTML to PDF

In [21]:

# def convert_html_to_pdf(source_html, output_filename):

#     result_file = open(output_filename, "w+b")

#     pisa_status = pisa.CreatePDF(
#             source_html,           
#             dest=result_file)           

#     result_file.close()           

#     return pisa_status.err

In [23]:
# convert_html_to_pdf(report, 'report.pdf')

In [ ]:
# ! open report.pdf